In [62]:
import numpy as np
import pandas as pd
data = pd.read_csv('../data/labeled.csv')

In [63]:
data.head(20)

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
0,1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.944160
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.327640
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.176700
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.757930
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,0,Blond,188,38022.162170
5,6,1991.0,male,60.0,Liberia,56080,senior security analyst,Bachelor,0,Blond,181,125809.990300
6,7,1982.0,NaN,71.0,State of Palestine,1205185,program associate,Bachelor,0,Black,174,150319.422600
7,8,2008.0,male,36.0,Israel,31419,retail specialist,NaN,0,Brown,190,11849.028760
8,9,2015.0,male,43.0,South Sudan,85333,intergovernmental outreach coordinator,No,0,Blond,189,48058.206070
9,10,2019.0,female,16.0,Cuba,1325383,lease analyst,Bachelor,1,Black,157,90940.689120


In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111993 entries, 0 to 111992
Data columns (total 12 columns):
Instance             111993 non-null int64
Year of Record       111552 non-null float64
Gender               104561 non-null object
Age                  111499 non-null float64
Country              111993 non-null object
Size of City         111993 non-null int64
Profession           111671 non-null object
University Degree    104623 non-null object
Wears Glasses        111993 non-null int64
Hair Color           104751 non-null object
Body Height [cm]     111993 non-null int64
Income in EUR        111993 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 10.3+ MB


In [71]:
data['Profession'].value_counts()

pipefitter                                  261
procedural justice coordinator              256
production support                          246
port marine engineer                        246
preplacement nurse practitioner             245
                                           ... 
actor                                         1
Brewery Manager                               1
accessibility outreach coordinator            1
blake fellow                                  1
assistant commissioner of administration      1
Name: Profession, Length: 1340, dtype: int64

In [72]:
data.describe()

,Instance,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR
count,111993.000000,111552.000000,111499.000000,1.119930e+05,111993.000000,111993.000000,1.119930e+05
mean,55997.000000,1999.421274,37.345304,8.388538e+05,0.500531,175.220192,1.092138e+05
std,32329.738686,11.576382,16.036694,2.196879e+06,0.500002,19.913889,1.498024e+05
min,1.000000,1980.000000,14.000000,7.700000e+01,0.000000,94.000000,-5.696906e+03
25%,27999.000000,1989.000000,24.000000,7.273400e+04,0.000000,160.000000,3.077169e+04
50%,55997.000000,1999.000000,35.000000,5.060920e+05,1.000000,174.000000,5.733917e+04
75%,83995.000000,2009.000000,48.000000,1.184501e+06,1.000000,190.000000,1.260936e+05
max,111993.000000,2019.000000,115.000000,4.999251e+07,1.000000,265.000000,5.285252e+06


In [61]:
## Pre-process data
# Replace useless values
data = data.fillna(0).replace('other', '0').replace('No', '0')

# Replace Degree values
print(' * Replacing \'University Degree\' and \'Gender\' values')
data = data.replace('No')
bins = { 
    'University Degree': {
        '0': 0,
       'Bachelor': 1, 
       'Master': 2,
       'PhD': 3
    },
    'Gender': {
        'male': 1,
        'female': -1,
        'other': 0,
    }
}
data = data.replace(to_replace=bins)

# Replace ['Country', 'Profession', 'Hair Color'] values
cols = ['Country', 'Profession', 'Hair Color']
for col in cols:
    print(' * Replacing \'%s\' values' % col)
    bins = {col: {}}
    curr = 0

    for val in data[col]:
        if val not in bins:
            bins[col][val] = int(curr)
            curr += 1

    data = data.replace(to_replace=bins)

data.head(10)

 * Replacing 'University Degree' and 'Gender' values
 * Replacing 'Country' values
 * Replacing 'Profession' values
 * Replacing 'Hair Color' values


,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
0,1,1997.0,0,41.0,111954,1239930,109246,1,0,111992,193,61031.94416
1,2,1996.0,0,41.0,111960,1603504,111537,2,0,111991,186,91001.32764
2,3,2018.0,0,28.0,111814,1298017,111010,3,1,111990,170,157982.17670
3,4,2006.0,0,33.0,111929,751903,107992,0,1,111991,171,45993.75793
4,5,2010.0,-1,46.0,111930,95389,111424,0,0,111992,188,38022.16217
5,6,1991.0,1,60.0,111973,56080,111791,1,0,111992,181,125809.99030
6,7,1982.0,0,71.0,111992,1205185,111957,1,0,111991,174,150319.42260
7,8,2008.0,1,36.0,111981,31419,111973,0,0,111990,190,11849.02876
8,9,2015.0,1,43.0,111948,85333,111395,0,0,111992,189,48058.20607
9,10,2019.0,-1,16.0,111929,1325383,111591,1,1,111991,157,90940.68912


In [54]:
data = data.astype(float)

ValueError: could not convert string to float: 'unknown'